In [24]:
import mysql.connector

def connet_db():
    # MySQL 데이터베이스 연결
    db_connection = mysql.connector.connect(
        host="localhost",  # MySQL 서버 주소
        user="root",       # MySQL 사용자 이름
        password="root1234",  # MySQL 사용자 비밀번호
        database="Final_project_team4"  # 사용할 데이터베이스 이름
    )
    return db_connection

def get_cursor(db_connection):
    # 커서 객체 생성
    cursor = db_connection.cursor()
    return cursor

# 데이터를 삽입할 함수 정의
def insert_specification_data(db_connection, cursor, product_main_name, product_id, spec_name, spac_value):
    # SQL 쿼리 준비
    query = f"INSERT INTO Specifications_{product_main_name} (product_id, spec_name, spac_value) VALUES (%s, %s, %s)"
    values = (product_id, spec_name, spac_value)

    # 쿼리 실행
    cursor.execute(query, values)

    # 변경 사항 저장
    db_connection.commit()
    print(f"Data inserted: {product_id}, {spec_name}, {spac_value}")


def find_product_id(cursor, product_name):
    # 특정 제품 이름으로 product_id 조회
    query = "SELECT product_id FROM products WHERE product_name = %s"
    cursor.execute(query, (product_name,))

    # 결과 가져오기
    result = cursor.fetchone()

    if result:
        product_id = result[0]
        print(f"The product_id for {product_name} is: {product_id}")
        return product_id
    else:
        raise print(f"No product found with the name '{product_name}'.")
        

##################################################################################################################################

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

def get_json_data(product_main_name):
    # 파일 경로를 지정하여 JSON 파일을 엽니다.
    file_path = f'naver_data_products/naver_api_response_{product_main_name}.json'

    # JSON 파일을 로드하는 방법
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def get_link(product_main_name:str, count:int):
    data = get_json_data(product_main_name)
    # 리턴할 링크 넘기기
    link = data[count]["link"]
    print(link)
    return link

def load_page(link):
    # 페이지 로드에 약 7~8초 걸림
    driver = webdriver.Chrome()
    url = link
    driver.get(url)
    driver.implicitly_wait(8)
    driver.refresh()
    driver.implicitly_wait(8)
    return driver

# 페이지 로드 대기
# time.sleep(5)

def click_butten(driver, xpath='//*[@id="container"]/div[2]/div[1]/div[3]/a'):
    # 버튼을 XPath로 찾아 클릭하기
    try:
        button = driver.find_element(By.XPATH, xpath)  # XPath로 버튼 요소 선택
        button.click()  # 버튼 클릭
        print("버튼을 클릭했습니다.")
    except Exception as e:
        raise print("버튼을 찾을 수 없거나 클릭할 수 없습니다.")

def get_data(driver):
    description_content=[]
    for j in range(1, 4):
        # 반복문 내에서 요소를 찾는 코드에 응답 대기 시간 적용
        try:
            for i in range(1, 12):
                # 최대 10초까지 대기
                row = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.XPATH, f'//*[@id="section_spec"]/div[{j}]/table[{i}]/tbody/tr'))
                )
                # description_content.append(meta_description.text.split())
                # tr 안의 th 요소 가져오기
                th_element = row.find_element(By.TAG_NAME, 'th')
                th_text = th_element.text

                # tr 안의 td 요소 가져오기
                td_element = row.find_element(By.TAG_NAME, 'td')
                td_text = td_element.text

                # th, td 각각 저장
                description_content.append({'header': th_text, 'value': td_text})
        except:
            pass
        if description_content:
            return description_content

def quit_driver(driver):
    # 드라이버 종료
    driver.quit()

def scroll_page(driver):
    # 현재 스크롤 위치 가져오기
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    # 현재 위치에서 y 좌표 200px 만큼 아래로 스크롤하기
    scroll_distance = 7000  # 이동할 거리
    new_scroll_position = current_scroll_position + scroll_distance
    driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")

def click_butten_smartstore(driver, xpath='//*[@id="_productFloatingTab"]/div/div[3]/ul/li[1]/a'):
    # 버튼을 XPath로 찾아 클릭하기
    try:
        button = driver.find_element(By.XPATH, xpath)  # XPath로 버튼 요소 선택
        button.click()  # 버튼 클릭
        print("버튼을 클릭했습니다.")
    except Exception as e:
        print("버튼을 찾을 수 없거나 클릭할 수 없습니다.")
    time.sleep(1)

def get_data_smartstore(driver):
    for j in range(3, 6):
        description_content=[]
        # 반복문 내에서 요소를 찾는 코드에 응답 대기 시간 적용
        try:
            # 최대 1초까지 대기
            for i in range(1, 12):
                row = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.XPATH, f'//*[@id="INTRODUCE"]/div/div[{j}]/div/div[2]/div/table/tbody/tr[{i}]'))
                )
                # description_content.append(meta_description.text)
                # tr 안의 th 요소 가져오기
                th_element = row.find_element(By.TAG_NAME, 'th')
                th_text = th_element.text

                # tr 안의 td 요소 가져오기
                td_element = row.find_element(By.TAG_NAME, 'td')
                td_text = td_element.text

                # th, td 각각 저장
                description_content.append({'header': th_text, 'value': td_text})
        except:
            pass
        if description_content:
            return description_content
        

def get_title(data, count):
    title = data[count]["title"].split()[:6]
    return " ".join(title)

def get_detail_spec_from_selenium(product_main_name, count = 0):
    link = get_link(product_main_name ,count)
    driver = load_page(link)
    data = get_json_data()
    product_name = get_title(data, count)
    if not "smartstore" in link:
        click_butten(driver=driver)
        description_content = get_data(driver)
        # description_content.insert(0, {'title' : product_name})
    elif "smartstore" in link:
        scroll_page(driver)
        click_butten_smartstore(driver)
        description_content = get_data_smartstore(driver)
        # description_content.insert(0, {'title' : product_name})
    else:
        pass
    quit_driver(driver)
    return product_name, description_content

def insert_selenium_data_to_mysql(product_main_name, product_name, description_content):
    # MySQL 데이터베이스 연결
    db_connection = connet_db()
    # 커서 객체 생성
    cursor = get_cursor(db_connection)
    # 제품 고유 id 추적
    product_id = find_product_id(cursor, product_name)

    # MySQL에 데이터 삽입
    table_name = f"Specifications_{product_main_name}"  # 테이블 이름 지정
    select_query = f"SELECT COUNT(*) FROM {table_name} WHERE product_id = %s AND spec_name = %s AND spec_value = %s"
    insert_query = f"INSERT INTO {table_name} (product_id, spec_name, spec_value) VALUES (%s, %s, %s)"

    for item in description_content:
        product_id = product_id
        header = item['header']
        value = item['value']
        values = (product_id, header, value)
        # 중복 데이터 확인
        cursor.execute(select_query, values)
        count = cursor.fetchone()[0]
        if count == 0:  # 중복되지 않은 경우에만 삽입
            cursor.execute(insert_query, values)  # 쿼리 실행
            # 변경 사항 커밋
            db_connection.commit()
            print(f"Inserted: {product_id} - {header} - {value}")
        else:
            print(f"Skipped (Duplicate): {product_id} - {header} - {value}")

    print("All data has been inserted into the database.")

    # 커서와 연결 종료
    cursor.close()
    db_connection.close()


In [28]:
from enum import Enum

class ProductMainName(Enum):
    SMARTPHONE = "smartphone"
    TABLETPC = "tabletpc"
    LAPTOP = "laptop"

# # Enum 값 출력
# print(f"Smartphone Key: {ProductMainName.SMARTPHONE.name}, Value: {ProductMainName.SMARTPHONE.value}")
# print(f"TabletPC Key: {ProductMainName.TABLETPC.name}, Value: {ProductMainName.TABLETPC.value}")
# print(f"Laptop Key: {ProductMainName.LAPTOP.name}, Value: {ProductMainName.LAPTOP.value}")


In [26]:
product_main_name = ProductMainName.SMARTPHONE.value
counts = get_json_data() # 100개 데이터 40분
for count in range(len(counts)):
    try:
        product_name, description_content = get_detail_spec_from_selenium(product_main_name, count)
        insert_selenium_data_to_mysql(product_main_name, product_name, description_content)
    except:
        print(f"Data has NoneType, So this count number({count}) is pass in the process")

https://search.shopping.naver.com/catalog/46670109618
버튼을 클릭했습니다.
The product_id for 삼성전자 갤럭시북4 NT750XGR-A51A is: 5
Skipped (Duplicate): 5 - 출시년도 - 2024년
Skipped (Duplicate): 5 - 종류 - 코어i5 13세대
Skipped (Duplicate): 5 - 램 - 16GB
Skipped (Duplicate): 5 - 화면크기 - 39.6cm(15.6인치)
Skipped (Duplicate): 5 - SSD - 256GB
Skipped (Duplicate): 5 - 블루투스 - 블루투스5.2
Skipped (Duplicate): 5 - 웹카메라 - 전면
Skipped (Duplicate): 5 - 보안기능 - TPM
Skipped (Duplicate): 5 - 스피커 - 스테레오스피커
Skipped (Duplicate): 5 - 색상 - 그레이
All data has been inserted into the database.
https://smartstore.naver.com/main/products/6174236911
버튼을 클릭했습니다.
The product_id for LG전자 울트라PC 15인치 사무용 인강용 업무용 is: 6
Skipped (Duplicate): 6 - CPU - 라이젠3-7330U
Skipped (Duplicate): 6 - CPU속도(기본클럭) - 2.3GHz
Skipped (Duplicate): 6 - 램 - 8GB
Skipped (Duplicate): 6 - 화면크기 - 39.6cm(15.6인치)
Skipped (Duplicate): 6 - 패널 - IPS패널(광시야각)
Skipped (Duplicate): 6 - 디스플레이 종류 - LCD
Skipped (Duplicate): 6 - AMD GPU - 라데온 Graphics
Skipped (Duplicate): 6 - SSD 인터페이스 - M.2
